In [2]:
#=======================================#
# Yes, this notebook is over-commented. #
#=======================================#

In [1]:
# Make notebook span entire screen, horizontally.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import gym
import numpy as np
import os
import tensorflow as tf

In [1]:
# TODO:
#  -> fix loss funtion
#  -> check reward processing

class PolicyAgent(object):
    def __init__(self, num_acts, num_features, num_units=10, learning_rate=0.001, decay=0.99):
        self.n_actions = num_acts
        self.n_features = num_features
        self.learning_rate = learning_rate
        self.decay = decay
        
        self.ep_obs, self.ep_acts, self.ep_rew = [], [], []
        
        # Build graph (i.e. internal neural network).
        self._build(num_units)
        
        # Create environment within which the graph will train.
        self.sess = tf.Session()
        
        # Evaluate an tf.Operation (returned by global_variables_initializer()) that initializes TRAINABLE VARIABLES.
        self.sess.run(tf.global_variables_initializer())
        
    def _build(self, num_units):
        # Set type/shape expectations for a tensor.
        self.obs      = tf.placeholder(tf.float32, (None, self.n_features))
        self.acts     = tf.placeholder(tf.int32, (None,))
        self.rew      = tf.placeholder(tf.float32, (None,))
        #self.l_rate   = tf.placeholder(tf.float32)
        self.training = tf.placeholder(tf.bool)
        
        h1 = tf.layers.dense(self.obs, num_units, activation=tf.nn.relu, kernel_initializer=tf.random_normal_initializer(), name='h1')
        h2 = tf.layers.dense(h1,       num_units, activation=tf.nn.relu, kernel_initializer=tf.random_normal_initializer(), name='h2')
        
        dropout = tf.layers.dropout(h2, training=self.training, name='dropout')
        
        # This needs to have linear activations to conform to the "unscaled log probabilities" requirement of `sparse_softmax_cross_entropy_loss_with_logits()`.
        out = tf.layers.dense(dropout, self.n_actions, kernel_initializer=tf.random_normal_initializer(), name='out')
        
        # Compute normalized probabilities associated with each action.
        self.probabilities = tf.nn.softmax(out)
        
        # Returns softmax cross entropy loss.
        print(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=out, labels=self.acts))
        
        loss = tf.reduce_mean(neg_log_prob * self.rew)
        
        # Set optimizer.
        self.train_op = tf.train.AdamOptimizer(self.learning_rate).minimize(loss)
    
    def choose_action(self, obs):
        # Compute probabilities associated with each action.
        prob_weights = self.sess.run(self.probabilities, feed_dict={
            self.obs:      np.array(obs).reshape(-1, self.n_features),
            self.training: False
        })
        
        # Choose action based on computed probabilities.
        return np.random.choice(range(prob_weights.shape[1]), p=prob_weights.ravel())
    
    def store(self, obs, action, reward):
        self.ep_obs.append(obs)
        self.ep_acts.append(action)
        self.ep_rew.append(reward)
        
    def process_rewards(self, episode):
        # Extract episode data:
        #  --> `action_history` is an ordered list of actions performed in the episode.
        #  --> `outcome` is `True` if the agent won or `False` if the agent lost.
        (action_history, outcome) = episode
        
        discounted_rewards = np.zeros_like(action_history)
        running_reward = 1 if outcome else -1
        
        for action_idx in reversed(range(len(action_history))):
            running_reward *= self.decay
            discounted_rewards[action_idx] = running_reward
        
        #for i in reversed(range(len(discounted_rewards))):
        #    running_reward *= self.decay
        #    running_reward += action_history[i]
        #    discounted_rewards[i] = running_reward
        
        discounted_rewards -= np.mean(discounted_rewards)
        if np.std(discounted_rewards) != 0:
            discounted_rewards /= np.std(discounted_rewards)
        return discounted_rewards
    
    def train(self, episode):
        discounted_rewards = self.process_rewards()
        
        self.sess.run(self.train_op, feed_dict={
            self.obs:      np.array(self.ep_obs).reshape(-1, self.n_features),
            self.acts:     np.array(self.ep_acts),
            self.rew:      np.array(discounted_rewards),
            self.training: True
        })
        
        # Clear episode-specific information.
        self.ep_obs, self.ep_acts, self.ep_rew = [], [], []
        
        return discounted_rewards    

In [ ]:
class AgentHandler(object):
    def __init__(self, agent, env, batch_size, num_batches, path='./.model.ckpt', verbosity=1):
        assert batch_size > 0, 'invalid batch_size'
        assert num_batches > 0, 'invalid num_batches'
        
        self.agent = agent
        self.env = env
        
        self.batch_size = batch_size
        self.num_batches = num_batches
        
        self.saver = tf.train.Saver() # saves all variables
        self.path = path
        
        self.verbosity = verbosity
        
    def save(self, path=self.path):
        self.saver.save(self.agent.sess, path)
        
    def load(self, path=self.path):
        self.saver.restore(self.agent.sess, path)
            
    def run(self):
        # Stores `(action_history, outcome)` tuples for `self.batch_size` episodes at a time.
        batch = []
        
        for episode in range(self.batch_size * self.num_batches):
            # Stores every action taken in current episode.
            action_history = []
            
            # Get observation of initial state of environment.
            obs_prev = self.env.reset()
            
            # Randomly choose first action in order to create first difference frame.
            obs_curr, reward, done, _ = self.env.step(env.action_space.sample())

            while not done:
                # Make difference frame.
                diff_frame, obs_prev = obs_curr - obs_prev, obs_curr
        
                self.env.render()
        
                # Agent chooses action based on difference frame.
                action = self.agent.choose_action(diff_frame)
        
                # Take action in environment.
                curr_obs, reward, done, _ = self.env.step(action)
                if self.verbosity >= 2:
                    print('Reward: %d' % reward)
                    
                # This is just exploratory/temporary.
                if not done: assert reward == 0, 'intermediate reward not always 0'
                
                action_history.append(action)

            if self.verbosity >= 1:
                print('Episode %d finished. ' % episode, end='')
                    
                if reward == 1:
                    if self.verbosity >= 1: print('Agent won.')
                    batch.append((action_history, True))
                elif reward == -1:
                    if self.verbosity >= 1: print('Agent lost.')
                    batch.append((action_history, False))
                else: assert False, 'unexpected reward'

            if len(batch) >= self.batch_size:
                # Train on batch and save agent.
                for ep in batch:
                    self.agent.train(ep)
                self.save()
                    
                # Start next batch.
                batch = []

In [9]:
env = gym.make('Breakout-v0') # RGB observation space

agent = PolicyAgent(num_acts=env.action_space.n,
                    num_features=env.observation_space.shape[0],
                    num_units=30,
                    learning_rate=0.001,
                    decay=0.99)

In [10]:
max_episodes = 1000000000000 # lmao

In [5]:
saver = tf.train.Saver()

In [6]:
saver.restore(agent.sess,'./.breakout/model.ckpt')

INFO:tensorflow:Restoring parameters from ./breakout_models/model.ckpt


In [9]:
save_path = saver.save(agent.sess, './.breakout/model.ckpt')

In [11]:
avg = []

cbuf = [None] * 20
c = 0

for episode_num in range(max_episodes):
    obs_prev = env.reset()
    
    # Randomly choose first action in order to create first difference frame.
    obs, rew, done, _ = env.step(env.action_space.sample())
    
    while True:
        # Make difference frame.
        diff_frame, obs_prev = obs - obs_prev, obs
        
        env.render()
        
        # Agent chooses action based on difference frame.
        action = agent.choose_action(diff_frame)
        
        # Take action in environment.
        obs, rew, done, _ = env.step(action)
        
        cbuf[c] = obs
        c = c + 1 if c < len(cbuf) - 1 else 0
        
        for i in range(cbuf):
            if i > 0:
                if not numpy.array_equal(cbuf[i], cbuf[i-1]):
                    stal
        
        # Remember stuff for delayed reward processing.
        agent.store(diff_frame, action, rew)
        
        if done:
            # Train model on episode.
            rewards = agent.train()
            
            avg.append(np.sum(rewards))
            
            # Save model every 100 episodes.
            if len(avg) >= 100:
                print(np.mean(avg), end=', ')
                save_path = saver.save(agent.sess, './breakout_models/model.ckpt')
            
            break

KeyboardInterrupt: 